# Sequential

In [1]:
import numpy as np
from math import sqrt

# heat sources used to create the border values
heat_sources = [ { 'x': 0.0, 'y': 0.0, 'range': 1.0, 'temp': 2.5 }, {'x': 0.5, 'y': 1.0, 'range': 1.0, 'temp': 2.5} ] 


# Apply the heat sources defined above. The handling of each border could be improved, but it works...
# Note that the order of the sources is important, values are overwritten, not modified
# There is no need to modify this function unless you find a bug
def apply_hs(m, hs, border):
    if border == 'top' or border == 'bottom':
        r = (0, m.shape[0])
    elif border == 'left' or border == 'right':
        r = (1, m.shape[0]-1)
    else:
        return

    for i in range(*r):
        if border == 'top':
            dist = sqrt(pow(i / (m.shape[0] - 1) - hs['x'], 2) + pow(hs['y'], 2))
        elif border == 'bottom':
            dist = sqrt(pow(i / (m.shape[0] - 1) - hs['x'], 2) + pow(1 - hs['y'], 2))
        elif border == 'left':
            dist = sqrt(pow(hs['x'], 2) + pow(i / (m.shape[0] - 1) - hs['y'], 2))
        elif border == 'right':
            dist = sqrt(pow(1 - hs['x'], 2) + pow(i / (m.shape[0] - 1) - hs['y'], 2))

        if dist <= hs['range']:
            if border == 'top':
                m[0, i] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'bottom':
                m[-1, i] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'left':
                m[i, 0] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'right':
                m[i,-1] += (hs['range'] - dist) / hs['range'] * hs['temp']


# Create a grid and apply the heat sources
def init(hs, n):
    num_p = n + 2

    m = np.zeros((num_p, num_p))
    for hs in heat_sources:
        apply_hs(m, hs, 'top')
        apply_hs(m, hs, 'bottom')
        apply_hs(m, hs, 'left')
        apply_hs(m, hs, 'right')

    return m


# Run a single iteration step, return the residual/error
def gauss_seidel_step(m):
    s = 0.0
    for i in range(1, m.shape[0]-1):
        for j in range(1, m.shape[1]-1):
            tmp = m[i, j]
            m[i, j] = (m[i, j-1] + m[i, j+1] + m[i-1, j] + m[i+1, j]) / 4
            diff = tmp - m[i, j]
            s += diff**2 * diff**2

    return s


# Run at most maxiter iterations. End if the residual/error is lower than the tolerance
def gauss_seidel(m, maxiter, tol):
    for i in range(maxiter):
        res = gauss_seidel_step(m)
        if res < tol:
            break
    return (res, i)


if __name__ == '__main__':
    m = init(heat_sources, 100)
    res, i = gauss_seidel(m, 25000, 0.00005)
    print(f'residual = {res} after {i} iterations')

residual = 4.593077140455239e-05 after 36 iterations


# Parallel

In [ ]:
import cupy as cp
import numpy as np
from math import sqrt

# heat sources used to create the border values
heat_sources = [{'x': 0.0, 'y': 0.0, 'range': 1.0, 'temp': 2.5},
                {'x': 0.5, 'y': 1.0, 'range': 1.0, 'temp': 2.5}]

# Apply the heat sources defined above
def apply_hs(m, hs, border):
    if border == 'top' or border == 'bottom':
        r = (0, m.shape[0])
    elif border == 'left' or border == 'right':
        r = (1, m.shape[0]-1)
    else:
        return

    for i in range(*r):
        if border == 'top':
            dist = sqrt(pow(i / (m.shape[0] - 1) - hs['x'], 2) + pow(hs['y'], 2))
        elif border == 'bottom':
            dist = sqrt(pow(i / (m.shape[0] - 1) - hs['x'], 2) + pow(1 - hs['y'], 2))
        elif border == 'left':
            dist = sqrt(pow(hs['x'], 2) + pow(i / (m.shape[0] - 1) - hs['y'], 2))
        elif border == 'right':
            dist = sqrt(pow(1 - hs['x'], 2) + pow(i / (m.shape[0] - 1) - hs['y'], 2))

        if dist <= hs['range']:
            if border == 'top':
                m[0, i] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'bottom':
                m[-1, i] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'left':
                m[i, 0] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'right':
                m[i,-1] += (hs['range'] - dist) / hs['range'] * hs['temp']


# Create a grid and apply the heat sources
def init(hs, n):
    num_p = n + 2

    m = np.zeros((num_p, num_p))
    for hs in heat_sources:
        apply_hs(m, hs, 'top')
        apply_hs(m, hs, 'bottom')
        apply_hs(m, hs, 'left')
        apply_hs(m, hs, 'right')

    return m

# Run a single iteration step, return the residual/error
def gauss_seidel_step_gpu(m):
    s = 0.0
    for i in range(1, m.shape[0] - 1):
        for j in range(1, m.shape[1] - 1):
            tmp = m[i, j]
            m[i, j] = (m[i, j - 1] + m[i, j + 1] + m[i - 1, j] + m[i + 1, j]) / 4
            diff = tmp - m[i, j]
            s += diff ** 2
    return s

# Run at most maxiter iterations. End if the residual/error is lower than the tolerance
def gauss_seidel_gpu(m, maxiter, tol):
    m_gpu = cp.array(m)
    for i in range(maxiter):
        res = gauss_seidel_step_gpu(m_gpu)
        if res < tol:
            break
    return (res, i)

if __name__ == '__main__':
    m = init(heat_sources, 100)
    res, i = gauss_seidel_gpu(m, 25000, 0.00005)
    print(f'residual = {res} after {i} iterations')

In [ ]:
import time
import numpy as np
from math import sqrt
from numba import cuda, prange, set_num_threads
import sys
from pathlib import Path
# workaround to be able to import common.heat
parent_dir = Path(__file__).resolve().parents[1]
sys.path.append(str(parent_dir))
import common.heat as heat

@cuda.jit
def update_cell(m, i, j):
    """Update a single cell and return the squared difference."""
    tmp = m[i, j]
    m[i, j] = (m[i, j-1] + m[i, j+1] + m[i-1, j] + m[i+1, j]) / 4
    diff = tmp - m[i, j]
    return diff ** 2

@cuda.jit
def update_cells_by_color(m, color):
    """Update either red or black cells based on the color parameter."""
    s = 0.0
    for i in range(1, m.shape[0]-1):
        for j in range(1, m.shape[1]-1):
            if (i + j) % 2 == color:
                s += update_cell(m, i, j)
    return s

@cuda.jit
def gauss_seidel_step_chessboard(m):
    (red_color_code, black_color_code) = (0, 1)
    s_red = update_cells_by_color(m, red_color_code) 
    s_black = update_cells_by_color(m, black_color_code)
    return s_red + s_black

def gauss_seidel(m, maxiter, tol):
    grid = cuda.to_device(m)

    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(m.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(m.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    for i in range(maxiter):
        res = gauss_seidel_step_chessboard[blockspergrid, threadsperblock](grid)
        if res < tol:
            break

    return (res, i)

def run_with_given_params(grid_size, maxiter, tol):
    m = heat.init(heat.heat_sources, grid_size)
    res, i = gauss_seidel(m, maxiter, tol)
    print(f'residual = {res} after {i} iterations')

def trigger_jit_compilation():
    run_with_given_params(10, 1, 1)

def run_and_test_scalability(maxiter, tol):
    grid_sizes = [100, 250, 500, 1000, 1500]
    thread_counts = [1, 2, 4, 8]
    test_scalability(grid_sizes, thread_counts, maxiter, tol)

def test_scalability(grid_sizes, thread_counts, maxiter, tol):
    for num_threads in thread_counts:
        set_num_threads(num_threads)
        print(f"Testing with {num_threads} threads...")
        for size in grid_sizes:
            m = heat.init(heat.heat_sources, size)
            start_time = time.time()
            res, i = gauss_seidel(m, maxiter, tol)
            end_time = time.time()
            
            execution_time = end_time - start_time
            print(f"  Grid Size: {size}x{size}, Execution Time: {execution_time:.2f} seconds, after {i} iterations, residual = {res}")

if __name__ == '__main__':
    maxiter = 25000
    tol = 0.00005
    # run_with_given_params(100, maxiter, tol)
    trigger_jit_compilation()
    run_and_test_scalability(maxiter, tol)